# Full Transfer Functions

In [1]:
%load_ext autoreload
import sys
sys.path.append("..")

In [55]:
%autoreload

import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10,10]


from astropy.io import fits

import numpy as np
import copy
import pickle

import darkhistory.physics as phys
import darkhistory.utilities as utils
import darkhistory.spec.spectools as spectools
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.transferfunclist as tflist
from darkhistory.spec.spectrum import Spectrum
from darkhistory.spec.spectra import Spectra
import darkhistory.history.tla as tla

from scipy.interpolate import interp1d

from tqdm import tqdm_notebook as tqdm

from darkhistory.electrons.ics.ics_spectrum import ics_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import engloss_spec
from darkhistory.electrons.ics.ics_cooling import get_ics_cooling_tf

import darkhistory.low_energy.lowE_photons as lowE_photons
import darkhistory.low_energy.lowE_electrons as lowE_electrons

# np.set_printoptions(threshold=np.nan)

## Photon Transfer Functions

In [20]:
user = 'hongwan'

highengphot_tflist_arr_raw = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_photspec_60eV_complete_coarse.raw", "rb"))
lowengphot_tflist_arr_raw  = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_lowengphotspec_60eV_complete_coarse.raw", "rb"))
lowengelec_tflist_arr_raw  = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_lowengelecspec_60eV_complete_coarse.raw", "rb"))
CMB_engloss_raw = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/CMB_engloss_60eV_complete_coarse.raw", "rb"))

In [23]:
xes = 0.5 + 0.5*np.tanh([-5., -4.1, -3.2, -2.3, -1.4, -0.5, 0.4, 1.3, 2.2, 3.1, 4])


In [58]:
%%prun
tf_interp = tflist.TransferFuncInterp(xes, highengphot_tflist_arr_raw)

[4.53978687e-05 2.74578156e-04 1.65880108e-03 9.95180187e-03
 5.73241759e-02 2.68941421e-01 6.89974481e-01 9.30861580e-01
 9.87871565e-01 9.97974680e-01 9.99664650e-01]
 

In [60]:
%%prun
tf_interp.get_tf(300, 0.5)

In [7]:
print(highengphot_tflist_arr_raw[0].grid_vals.shape)
a = np.atleast_3d(
                np.stack([tf.grid_vals for tf in highengphot_tflist_arr_raw[0]])
            )
print(a.shape)
print(highengphot_tflist_arr_raw[0][0])
print(highengphot_tflist_arr_raw[0][0].grid_vals.shape)

(51, 325, 500)
(51, 325, 500)
(325, 500)


In [8]:
b = highengphot_tflist_arr_raw[0]
print(b.grid_vals.shape)
b.transpose()
print(b.grid_vals.shape)

(51, 325, 500)
(325, 51, 500)


In [9]:
test1 = Spectrum(np.array([1,2]), np.array([3,4]), spec_type = 'N')
test2 = Spectrum(np.array([1,2]), np.array([3,4]), spec_type = 'N')
a = tf.TransFuncAtRedshift([test1, test2])

In [10]:
a.grid_vals

array([[3, 4],
       [3, 4]])

In [16]:
b = tf.TransFuncAtRedshift(np.array([[3,4,5], [3,4,5]]), eng = np.array([1,2,3]), in_eng = np.array([1, 2]))

In [17]:
print(b.grid_vals)
print(b.eng)
print(b.in_eng)
print(b.rs)
print(b.N_underflow)
print(b.eng_underflow)

[[3 4 5]
 [3 4 5]]
[1 2 3]
[1 2]
[-1. -1.]
[0 0]
[0 0]


In [18]:
b.eng.size

3